# 1. 서울 열린데이터 광장

서울 열린데이터 광장(Seoul Open Data Plaza)은 서울시에서 운영하는 공공데이터 개방 플랫폼
https://data.seoul.go.kr/
https://data.seoul.go.kr/dataList/OA-15493/A/1/datasetView.do

# 2. 서울시 공공자전거 실시간 대여정보
	4b5661637164656a34375359466168
  http://openapi.seoul.go.kr:8088/4b5661637164656a34375359466168/json/bikeList/1/5/

In [2]:
import requests
import folium
import json
import pandas as pd
import warnings

warnings.filterwarnings('ignore')


In [3]:
baseURL = 'http://openapi.seoul.go.kr:8088/4b5661637164656a34375359466168/json/bikeList/1/5/'
req = requests.get(baseURL)
req

<Response [200]>

In [11]:
json_data = json.loads(req.text)
json_data

json_data['rentBikeStatus']['RESULT']['CODE']

'INFO-000'

In [12]:
json_data.get("rentBikeStatus", {}).get("RESULT", {}).get("CODE", "")

'INFO-000'

In [32]:
def fetch_bike_data() :
  base_url = baseURL = 'http://openapi.seoul.go.kr:8088/4b5661637164656a34375359466168/json/bikeList/'
  start = 1
  end = 1000
  step = 1000
  data_frames = []

  while True:
    url = f"{base_url}{start}/{end}/"
    request = requests.get(url)
    if request.status_code != 200 :
      print(f"status_code : {request.status_code}")
      break
    json_data = request.json()
    try :
      rent_bike_status = json_data['rentBikeStatus']
      result_code = rent_bike_status['RESULT']['CODE']

    except KeyError:
      print("KeyError")
      break

    if result_code == 'INFO-200' :
      print('더이상 데이터가 없음')
      break
    elif result_code == 'INFO-000' :
      print(f"시작은 {start}, 끝은 {end}")
      try :
        bike_data = rent_bike_status['row']
        if bike_data :
          df = pd.DataFrame(bike_data)
          data_frames.append(df)
      except KeyError :
        print("KeyError")
        break
    else :
      print(f"result code : {result_code}")

    start += step
    end += step

  if data_frames :
    final_df = pd.concat(data_frames, ignore_index=True)
    return final_df
  else :
    return pd.DataFrame()


In [33]:
bike_data_df = fetch_bike_data()

시작은 1, 끝은 1000
시작은 1001, 끝은 2000
시작은 2001, 끝은 3000
KeyError


In [ ]:
'''
rackTotCnt  거치대개수
parkingBikeTotCnt   자전거주차총건수
shared  거치율
stationLatitude 위도
stationLongitude    경도
stationId   대여소ID
stationName 대여소이름
'''

In [36]:
bike_data_df['stationLatitude'] = bike_data_df['stationLatitude'].astype(float)
bike_data_df['stationLongitude'] = bike_data_df['stationLongitude'].astype(float)

bike_map = folium.Map(location =[bike_data_df['stationLatitude'].mean(), bike_data_df['stationLongitude'].mean()], zoom_start = 12)
for i, data in bike_data_df.iterrows():
  popup_str = '{} 자전거 주차 총 건수 : {}대 '.format(data['stationName'] , data['parkingBikeTotCnt'])
  popup = folium.Popup(popup_str, max_width=600)
  folium.Marker(location = [data['stationLatitude'], data['stationLongitude']], popup=popup).add_to(bike_map)

In [37]:
bike_map